# Model to STM32 Conversion using ST Edge AI

This notebook demonstrates how to convert machine learning models to STM32-compatible code using ST Edge AI tools.

We'll convert:
1. CenterFace TFLite model for face detection
2. MobileFaceNet ONNX model for face recognition

In [2]:
import os
import subprocess
import json
from pathlib import Path

# Use STM32CubeMX version that works (v2.1.0 instead of v2.2.0)
STEDGEAI_PATH = '/home/vboxuser/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/10.1.0/Utilities/linux/stedgeai'

# Test if stedgeai is accessible at the STM32CubeMX path
if os.path.exists(STEDGEAI_PATH):
    print(f"✅ stedgeai found at: {STEDGEAI_PATH}")
    
    # Test if it's executable and check for AVX requirements
    try:
        result = subprocess.run([STEDGEAI_PATH, '--help'], capture_output=True, text=True, timeout=5)
        print("✅ stedgeai is executable and responding")
        # Check version
        if 'v2.1.0' in result.stdout:
            print("✅ Using STM32CubeMX compatible version (v2.1.0)")
        else:
            print("ℹ️  Version info not found in help output")
    except (subprocess.CalledProcessError, subprocess.TimeoutExpired) as e:
        print(f"⚠️  stedgeai found but may have issues: {e}")
    except Exception as e:
        if "AVX" in str(e) or "SIGILL" in str(e):
            print("❌ AVX instruction set not available - this is likely the issue")
            print("Solutions:")
            print("1. Enable AVX in VM settings (see instructions)")
            print("2. Use a different machine with AVX support")
            print("3. Try running stedgeai with compatibility flags")
else:
    print(f"❌ stedgeai not found at: {STEDGEAI_PATH}")
    print("Please verify the installation path")

# Check CPU capabilities
print("\n=== CPU Capabilities Check ===")
try:
    result = subprocess.run(['lscpu'], capture_output=True, text=True)
    cpu_info = result.stdout
    if 'avx' in cpu_info.lower():
        print("✅ AVX support detected")
    else:
        print("❌ No AVX support detected")
        print("This VM needs AVX enabled in hypervisor settings")
except:
    print("Could not check CPU capabilities")

# Set up paths
models_dir = Path('./onnx_tflite_src')
output_dir = Path('./converted_models')
output_dir.mkdir(exist_ok=True)

# Model paths
centerface_model = models_dir / 'centerface.tflite'
mobilefacenet_model = models_dir / 'mobilefacenet_int8_faces.onnx'

print(f"CenterFace model exists: {centerface_model.exists()}")
print(f"MobileFaceNet model exists: {mobilefacenet_model.exists()}")

✅ stedgeai found at: /home/vboxuser/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/10.1.0/Utilities/linux/stedgeai
✅ stedgeai is executable and responding
✅ Using STM32CubeMX compatible version (v2.1.0)

=== CPU Capabilities Check ===
✅ AVX support detected
CenterFace model exists: True
MobileFaceNet model exists: True


## Configuration for ST Edge AI

Create configuration files for optimized STM32 generation

## Memory Layout Strategy

The two models use different memory pool configurations to avoid conflicts in external flash:

**Memory Pool Assignments:**
- **CenterFace (model1.mpool)**: External flash starts at `0x71000000` (Face Detection)
- **MobileFaceNet (model2.mpool)**: External flash starts at `0x72000000` (Face Recognition)


**Flash Memory Layout:**
```
0x70000000 - 0x700FFFFF: Bootloader code (1MB)
0x70100000 - 0x709FFFFF: Application code (8MB)
0x70A00000 - 0x70FFFFFF: Reserved space (6MB)
0x71000000 - 0x71FFFFFF: Face Detection model data (16MB)
0x72000000 - 0x72FFFFFF: Face Recognition model data (16MB)
0x73000000 - 0x74FFFFFF: Available for other uses (32MB)
```

This layout ensures:
- No model data overwrites the bootloader or application
- Both models can coexist without conflicts
- Efficient memory utilization on STM32N6 with external flash"

In [3]:
# Create neural art configuration for face detection (CenterFace)
# Uses model1.mpool with external flash at 0x71000000
face_detection_config = {
    "Globals": {},
    "Profiles": {
        "centerface": {
            "memory_pool": "./memory_pools/model1.mpool",
            "options": "-O0 --all-buffers-info --mvei --cache-maintenance --Oalt-sched  --enable-virtual-mem-pools --Omax-ca-pipe 4 --Ocache-opt --Os --enable-epoch-controller"
        }
    }
}

# Create neural art configuration for face recognition (MobileFaceNet)
# Uses model2.mpool with external flash at 0x72000000
face_recognition_config = {
    "Globals": {},
    "Profiles": {
        "mobilefacenet": {
            "memory_pool": "./memory_pools/model2.mpool",
            "options": "-O0 --all-buffers-info --mvei --cache-maintenance --Oalt-sched --enable-virtual-mem-pools --Omax-ca-pipe 4 --Ocache-opt --Os --enable-epoch-controller"
        }
    }
}

# Save configurations
with open('face_detection_config.json', 'w') as f:
    json.dump(face_detection_config, f, indent=4)

with open('face_recognition_config.json', 'w') as f:
    json.dump(face_recognition_config, f, indent=4)

print("Configuration files created with proper memory pool assignments:")
print("- CenterFace (Face Detection): model1.mpool (external flash @ 0x71000000)")
print("- MobileFaceNet (Face Recognition): model2.mpool (external flash @ 0x72000000)")

Configuration files created with proper memory pool assignments:
- CenterFace (Face Detection): model1.mpool (external flash @ 0x71000000)
- MobileFaceNet (Face Recognition): model2.mpool (external flash @ 0x72000000)


## Convert CenterFace TFLite Model

In [4]:
import os
import subprocess
from IPython.display import clear_output

def run_stedgeai_conversion(model_path,
                            output_name,
                            target="stm32n6",
                            input_data_type="uint8",
                            neural_art_config="",
                            profile_config=""):
    """Run ST Edge AI conversion but show a single, updating progress bar line in Jupyter."""
    if not os.path.exists(STEDGEAI_PATH):
        print(f"Error: stedgeai not found at {STEDGEAI_PATH}")
        return False

    # disable any built‑in progress suppression
    env = os.environ.copy()
    env.pop("TQDM_DISABLE", None)

    cmd = [
        STEDGEAI_PATH, "generate",
        "--name",           str(output_name),
        "--model",          str(model_path),
        "--target",         target,
        "--st-neural-art",  f"{profile_config}@{neural_art_config}",
        "--input-data-type",input_data_type,
        "--output",         str(output_dir)
    ]

    print(f"Running: {' '.join(cmd)}\n")

    # Launch the process and stream its combined stdout+stderr
    proc = subprocess.Popen(cmd,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.STDOUT,
                            text=True,
                            bufsize=1,
                            env=env)

    try:
        for raw_line in proc.stdout:
            # strip trailing newline but keep any leading 'PASS:' or similar
            line = raw_line.rstrip("\n")
            # overwrite previous output instead of appending
            clear_output(wait=True)
            print(line)

        proc.wait()
        if proc.returncode != 0:
            print(f"\n❌ Conversion failed (exit code {proc.returncode})")
            return False

        print("\n✅ Conversion completed successfully.")
        return True

    except Exception as e:
        proc.kill()
        clear_output(wait=True)
        print(f"\n❌ Error during conversion: {e}")
        return False


In [5]:
# Convert CenterFace model
print("Converting CenterFace TFLite model...")
centerface_success = run_stedgeai_conversion(
    centerface_model, 
    'face_detection',
    target="stm32n6",
    input_data_type="float32",
    neural_art_config = "face_detection_config.json",
    profile_config = "centerface"
)

if centerface_success:
    print("✅ CenterFace model conversion completed successfully")
else:
    print("❌ CenterFace model conversion failed")

elapsed time (generate): 17.781s

✅ Conversion completed successfully.
✅ CenterFace model conversion completed successfully


## Convert MobileFaceNet ONNX Model

In [6]:
# Convert MobileFaceNet model
print("Converting MobileFaceNet ONNX model...")
mobilefacenet_success = run_stedgeai_conversion(
    mobilefacenet_model, 
    'face_recognition',
    target="stm32n6",
    input_data_type="float32",
    neural_art_config = "face_recognition_config.json",
    profile_config = "mobilefacenet"
)

if mobilefacenet_success:
    print("✅ MobileFaceNet model conversion completed successfully")
else:
    print("❌ MobileFaceNet model conversion failed")

elapsed time (generate): 19.456s

✅ Conversion completed successfully.
✅ MobileFaceNet model conversion completed successfully


## Summary and Next Steps

In [7]:
print("\n" + "="*50)
print("CONVERSION SUMMARY")
print("="*50)

print(f"CenterFace TFLite → STM32: {'✅ Success' if centerface_success else '❌ Failed'}")
print(f"MobileFaceNet ONNX → STM32: {'✅ Success' if mobilefacenet_success else '❌ Failed'}")

print("\nGenerated files are organized in the ./stm32_output directory")
print("\nNext steps:")
print("1. Review the generated network.c and network.h files")
print("2. Integrate the models into your STM32 project")
print("3. Configure memory pools based on the .mpool files")
print("4. Test the models on your target STM32 hardware")

# List generated files
print("\nGenerated files:")
for item in output_dir.rglob('*'):
    if item.is_file():
        print(f"  {item.relative_to(output_dir)}")


CONVERSION SUMMARY
CenterFace TFLite → STM32: ✅ Success
MobileFaceNet ONNX → STM32: ✅ Success

Generated files are organized in the ./stm32_output directory

Next steps:
1. Review the generated network.c and network.h files
2. Integrate the models into your STM32 project
3. Configure memory pools based on the .mpool files
4. Test the models on your target STM32 hardware

Generated files:
  mobilefacenet_int8_faces_OE_3_2_0.onnx
  LICENSE.txt
  face_recognition_c_info.json
  face_recognition.h
  centerface_OE_3_2_0_Q.json
  face_recognition.c
  face_recognition_atonbuf.xSPI2.raw
  face_detection.c
  centerface_OE_3_2_0.onnx
  face_recognition_generate_report.txt
  face_detection_c_info.json
  face_detection.h
  face_detection_atonbuf.xSPI2.raw
  mobilefacenet_int8_faces_OE_3_2_0_Q.json
  face_detection_generate_report.txt
  face_recognition_ecblobs.h
  face_detection_ecblobs.h
  code/face_recognition.h
  code/face_recognition.c
  code/face_detection.c
  code/face_detection.h
  code/fac

## Post-processing and File Organization

In [8]:
import shutil
def organize_output_files(model_name):
    st_ai_output = Path('converted_models')

    if st_ai_output.exists():
        code_dir = Path(st_ai_output,'code')
        binaries_dir = Path(st_ai_output, 'binaries')
        code_dir.mkdir(exist_ok=True)
        binaries_dir.mkdir(exist_ok=True)
        # Copy header and C files
        patterns = [f'{model_name}*.c', f'{model_name}*.h', f'{model_name}*_ecblobs.h', 'f{model_name}*_data.h']

        for pattern in patterns:
            for src_file in st_ai_output.glob(pattern):
                dst_file = code_dir / src_file.name
                shutil.move(src_file, dst_file)
                print(f"moved {src_file.name} to {dst_file}")

        # Handle raw binary file
        binary_files = list(st_ai_output.glob(f'{model_name}*.raw'))
        if binary_files:
            
            binary_file = binary_files[0]
            print(binary_file)
            bin_output = binaries_dir / f'{model_name}_data.bin'
            print(bin_output)
            hex_output = binaries_dir / f'{model_name}_data.hex'
            print(hex_output)
            shutil.copy(binary_file, bin_output)
            print(f"Copied binary: {binary_file.name} to {bin_output}")

            # Set address
            address_map = {
                'face_detection': '0x71000000',
                'face_recognition': '0x72000000',
            }
            address = address_map.get(model_name, '0x70380000')

            try:
                subprocess.run([
                    'arm-none-eabi-objcopy', '-I', 'binary', str(bin_output),
                    '--change-addresses', address, '-O', 'ihex', str(hex_output)
                ], check=True)
                print(f"Generated HEX file: {hex_output} at address {address}")
            except subprocess.CalledProcessError as e:
                print(f"Warning: HEX generation failed: {e}")
    else:
        print("Warning: st_ai_output directory not found")

    return st_ai_output
    

In [9]:
if centerface_success:
    centerface_dir = organize_output_files('face_detection')
    print(f"CenterFace files organized in: {centerface_dir}")


moved face_detection.c to converted_models/code/face_detection.c
moved face_detection.h to converted_models/code/face_detection.h
moved face_detection_ecblobs.h to converted_models/code/face_detection_ecblobs.h
converted_models/face_detection_atonbuf.xSPI2.raw
converted_models/binaries/face_detection_data.bin
converted_models/binaries/face_detection_data.hex
Copied binary: face_detection_atonbuf.xSPI2.raw to converted_models/binaries/face_detection_data.bin
Generated HEX file: converted_models/binaries/face_detection_data.hex at address 0x71000000
CenterFace files organized in: converted_models


In [10]:
if mobilefacenet_success:
    mobilefacenet_dir = organize_output_files('face_recognition')
    print(f"MobileFaceNet files organized in: {mobilefacenet_dir}")

moved face_recognition.c to converted_models/code/face_recognition.c
moved face_recognition.h to converted_models/code/face_recognition.h
moved face_recognition_ecblobs.h to converted_models/code/face_recognition_ecblobs.h
converted_models/face_recognition_atonbuf.xSPI2.raw
converted_models/binaries/face_recognition_data.bin
converted_models/binaries/face_recognition_data.hex
Copied binary: face_recognition_atonbuf.xSPI2.raw to converted_models/binaries/face_recognition_data.bin
Generated HEX file: converted_models/binaries/face_recognition_data.hex at address 0x72000000
MobileFaceNet files organized in: converted_models
